In [2]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath('../'))
from src.hopsworks_connections import upload_data, pull_model, pull_data

# Print the current working directory
current_working_directory = os.getcwd()
print("Current Working Directory:", current_working_directory)

/Users/davydsadovskyy/GitBackedShit/crypto-prediction/venv_crypto_prediction/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Current Working Directory: /Users/davydsadovskyy/GitBackedShit/crypto-prediction/notebooks


### Get the latest day of ethereum data

In [3]:
# we'll later also need to update the day's before yesterday data for tmw (tmw_avg_high_close, tmw_percent_increase)
eth_last_2_days = pull_data('eth_ohlc_transformed', 1, 'eth_ohlc_transformed_view', 1)[-2:]
eth_yesterday = eth_last_2_days[-1:]

eth_yesterday

Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.
Feature view already existed. Skip creation.
Finished: Reading data from Hopsworks, using ArrowFlight (1.58s) 


,date,open,high,low,close,volume,volume_eth,market_cap,tmw_avg_high_close,tmw_percent_increase,...,october,november,december,monday,tuesday,wednesday,thursday,friday,saturday,sunday
3063,2024-04-23 00:00:00+00:00,3200.817053,3223.265627,3182.763954,3182.763954,1.115352e+10,3.481270e+06,3.897252e+11,NaN,NaN,...,0,0,0,0,1,0,0,0,0,0


### Get model from our Hopsworks model registry

In [4]:
MODEL_NAME = 'random_forest_ohlc' # this one predicts tmw_1_0_percent_increase_binary

# ***** later implement code that checks the latest model version and pulls that one
# def pull_model(model_name, model_version):
model = pull_model(MODEL_NAME, 1)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.


### Define predictors, the input data points, and get a prediction for tmw_1_0_percent_increase_binary

In [5]:
predictors = []
periods = [2, 5, 10, 25, 50, 100]
percent_increase_counts = ['1_0', '1_25', '1_5', '1_75', '2_0', '2_25', '2_5', '2_75', '3_0']
for period in periods:
    for indicator in ['ema', 'rsi', 'sma']:
        predictors.append(f'{indicator}_{period}')
    for percent in percent_increase_counts:
        predictors.append(f'last_{period}_day_{percent}_percent_increase_count')
months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
predictors.extend(months)
weekdays = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
predictors.extend(weekdays)

x0 = eth_yesterday[predictors]
x0

,ema_2,rsi_2,sma_2,last_2_day_1_0_percent_increase_count,last_2_day_1_25_percent_increase_count,last_2_day_1_5_percent_increase_count,last_2_day_1_75_percent_increase_count,last_2_day_2_0_percent_increase_count,last_2_day_2_25_percent_increase_count,last_2_day_2_5_percent_increase_count,...,october,november,december,monday,tuesday,wednesday,thursday,friday,saturday,sunday
3063,1.000309,64.876621,0.997336,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0


In [6]:
prediction = model.predict(x0)
prediction

array([0.])

In [7]:
eth_yesterday['pred_tmw_1_0_percent_increase_binary'] = int(prediction[0])
eth_yesterday['pred_tmw_1_0_percent_increase_binary'] = eth_yesterday['pred_tmw_1_0_percent_increase_binary'].astype('Int64')

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### Update our Hopsworks predictions data with this new prediction

In [8]:
previous_predictions = pull_data('eth_ohlc_predictions', 1, 'eth_ohlc_predictions', 1)
previous_predictions

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.
Feature view already existed. Skip creation.
Finished: Reading data from Hopsworks, using ArrowFlight (0.62s) 


,date,open,low,high,close,tmw_avg_high_close,tmw_percent_increase,pred_tmw_1_0_percent_increase_binary,tmw_1_0_percent_increase_binary,tmw_1_25_percent_increase_binary,tmw_1_5_percent_increase_binary,tmw_1_75_percent_increase_binary,tmw_2_0_percent_increase_binary,tmw_2_25_percent_increase_binary,tmw_2_5_percent_increase_binary,tmw_2_75_percent_increase_binary,tmw_3_0_percent_increase_binary,tmw_positive_percent_increase_binary
807,2015-11-15 00:00:00+00:00,0.891200,0.875000,0.921500,0.906400,0.936850,0.032503,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1226,2015-11-16 00:00:00+00:00,0.906200,0.892000,0.944700,0.929000,1.020000,0.089216,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2433,2015-11-17 00:00:00+00:00,0.924900,0.905800,1.030000,1.010000,1.000000,-0.010000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1969,2015-11-18 00:00:00+00:00,0.990000,0.940500,1.010000,0.990000,0.982750,-0.007377,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
860,2015-11-19 00:00:00+00:00,0.988700,0.937500,1.010000,0.955500,0.940300,-0.016165,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2752,2024-04-18 00:00:00+00:00,2986.000000,2960.000000,3087.000000,3064.000000,3101.000000,0.011932,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2794,2024-04-19 00:00:00+00:00,3061.000000,2879.000000,3117.000000,3085.000000,3160.500000,0.023889,0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
2706,2024-04-20 00:00:00+00:00,3085.000000,3025.000000,3166.000000,3155.000000,3168.500000,0.004261,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
952,2024-04-21 00:00:00+00:00,3153.000000,3120.000000,3191.000000,3146.000000,3215.178412,0.021516,0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0


In [9]:
vars = ['date', 'open', 'low', 'high', 'close', 'tmw_avg_high_close', 'tmw_percent_increase', 'pred_tmw_1_0_percent_increase_binary', 
        'tmw_1_0_percent_increase_binary', 'tmw_1_25_percent_increase_binary', 'tmw_1_5_percent_increase_binary', 
        'tmw_1_75_percent_increase_binary', 'tmw_2_0_percent_increase_binary', 'tmw_2_25_percent_increase_binary', 
        'tmw_2_5_percent_increase_binary', 'tmw_2_75_percent_increase_binary', 'tmw_3_0_percent_increase_binary', 
        'tmw_positive_percent_increase_binary']

eth_yesterday = eth_yesterday[vars]

eth_yesterday

,date,open,low,high,close,tmw_avg_high_close,tmw_percent_increase,pred_tmw_1_0_percent_increase_binary,tmw_1_0_percent_increase_binary,tmw_1_25_percent_increase_binary,tmw_1_5_percent_increase_binary,tmw_1_75_percent_increase_binary,tmw_2_0_percent_increase_binary,tmw_2_25_percent_increase_binary,tmw_2_5_percent_increase_binary,tmw_2_75_percent_increase_binary,tmw_3_0_percent_increase_binary,tmw_positive_percent_increase_binary
3063,2024-04-23 00:00:00+00:00,3200.817053,3182.763954,3223.265627,3182.763954,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Check if the date from eth_yesterday is not in previous_predictions
if eth_yesterday['date'].iloc[0] not in previous_predictions['date'].values:

    # Append eth_yesterday to previous_predictions
    previous_predictions = pd.concat([previous_predictions, eth_yesterday], ignore_index=True)

    # Update the last row of previous_predictions with values from eth_last_2_days
    previous_predictions.at[previous_predictions.index[-2], 'tmw_avg_high_close'] = eth_last_2_days['tmw_avg_high_close'].iloc[-2]
    previous_predictions.at[previous_predictions.index[-2], 'tmw_percent_increase'] = eth_last_2_days['tmw_percent_increase'].iloc[-2]
    previous_predictions.at[previous_predictions.index[-2], 'tmw_positive_percent_increase_binary'] = eth_last_2_days['tmw_positive_percent_increase_binary'].iloc[-2]
    previous_predictions.at[previous_predictions.index[-2], 'tmw_1_0_percent_increase_binary'] = eth_last_2_days['tmw_1_0_percent_increase_binary'].iloc[-2]
    previous_predictions.at[previous_predictions.index[-2], 'tmw_1_25_percent_increase_binary'] = eth_last_2_days['tmw_1_25_percent_increase_binary'].iloc[-2]
    previous_predictions.at[previous_predictions.index[-2], 'tmw_1_5_percent_increase_binary'] = eth_last_2_days['tmw_1_5_percent_increase_binary'].iloc[-2]
    previous_predictions.at[previous_predictions.index[-2], 'tmw_1_75_percent_increase_binary'] = eth_last_2_days['tmw_1_75_percent_increase_binary'].iloc[-2]
    previous_predictions.at[previous_predictions.index[-2], 'tmw_2_0_percent_increase_binary'] = eth_last_2_days['tmw_2_0_percent_increase_binary'].iloc[-2]
    previous_predictions.at[previous_predictions.index[-2], 'tmw_2_75_percent_increase_binary'] = eth_last_2_days['tmw_2_75_percent_increase_binary'].iloc[-2]
    previous_predictions.at[previous_predictions.index[-2], 'tmw_3_0_percent_increase_binary'] = eth_last_2_days['tmw_3_0_percent_increase_binary'].iloc[-2]

else:
    print('eth_yesterday was already in previous_predictions')

### Put back into Hopsworks

In [11]:
# make sure some of the column types are what hopsworks wants
previous_predictions['date'] = pd.to_datetime(previous_predictions['date']).dt.tz_localize(None)
columns = [
    'tmw_1_0_percent_increase_binary',
    'tmw_1_25_percent_increase_binary',
    'tmw_1_5_percent_increase_binary',
    'tmw_1_75_percent_increase_binary',
    'tmw_2_0_percent_increase_binary',
    'tmw_2_25_percent_increase_binary',
    'tmw_2_5_percent_increase_binary',
    'tmw_2_75_percent_increase_binary',
    'tmw_3_0_percent_increase_binary',
    'tmw_positive_percent_increase_binary'
]
previous_predictions[columns] = previous_predictions[columns].astype('Int64')

upload_data(previous_predictions, 'eth_ohlc_predictions', 1)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.


Uploading Dataframe: 100.00% |██████████| Rows 3064/3064 | Elapsed Time: 00:04 | Remaining Time: 00:00


Launching job: eth_ohlc_predictions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/582805/jobs/named/eth_ohlc_predictions_1_offline_fg_materialization/executions
